In [1]:
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle fastai py7zr scikit-learn

In [2]:
from fastkaggle import *

In [3]:
# for working with paths in Python, I recommend using `pathlib.Path`
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not iskaggle and not cred_path.exists():
    creds = ''
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [4]:
comp = 'kkbox-music-recommendation-challenge'

path = setup_comp(comp, install='fastai')

In [5]:
path

Path('kkbox-music-recommendation-challenge')

In [6]:
from fastai.tabular.all import *
from fastai.collab import *

path.ls()

(#12) [Path('kkbox-music-recommendation-challenge/train.csv'),Path('kkbox-music-recommendation-challenge/test.csv.7z'),Path('kkbox-music-recommendation-challenge/test.csv'),Path('kkbox-music-recommendation-challenge/songs.csv.7z'),Path('kkbox-music-recommendation-challenge/sample_submission.csv.7z'),Path('kkbox-music-recommendation-challenge/train.csv.7z'),Path('kkbox-music-recommendation-challenge/sample_submission.csv'),Path('kkbox-music-recommendation-challenge/members.csv.7z'),Path('kkbox-music-recommendation-challenge/song_extra_info.csv'),Path('kkbox-music-recommendation-challenge/songs.csv')...]

In [7]:
def is_extracted(file):
    return os.path.isfile(os.path.splitext(file)[0])

for file in path.ls(file_exts=".7z"):
    if not is_extracted(file):
        !py7zr x {str(file)} {str(path)}

In [8]:
path.ls(file_exts=".csv")

(#6) [Path('kkbox-music-recommendation-challenge/train.csv'),Path('kkbox-music-recommendation-challenge/test.csv'),Path('kkbox-music-recommendation-challenge/sample_submission.csv'),Path('kkbox-music-recommendation-challenge/song_extra_info.csv'),Path('kkbox-music-recommendation-challenge/songs.csv'),Path('kkbox-music-recommendation-challenge/members.csv')]

In [9]:
songs = pd.read_csv(path/'train.csv', usecols=('msno', 'song_id', 'target'))

songs.head()

,msno,song_id,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1


In [10]:
dls = CollabDataLoaders.from_df(songs, bs=64)
dls.show_batch()

,msno,song_id,target
0,BrFfyWCsRpGkKZXmmRUSaSjiSsf7inBBRD1AmoWMbZc=,ajjUpdQjWCwzCHNd6C0hB3Fz8BIPGjSy6IT35+UWD1I=,0
1,AzZEHNKrDpTqTDYYdsho+24eUlK0bigsNMEjo4EfFzU=,lQ3sqM5nGkBWG1JIunEHbYkG3Z0EN3t74fyqR9/OSgo=,0
2,iJ4dVFSgX5mh8bbYKo5dv3OoF+3Bx9AwIplTDZ2sDdk=,+BzA5HhQIWajCEKnb532d8f4Lcy1IcuaXiI4xYAA0ek=,1
3,ON5iFBZdRcjplFOtqdCY+Dj5gShVAMy99o1hDvWo7bA=,g2TxUJZJZW/LNaryWMEKnwjcQL9o7lTW5pMdjmjLynE=,1
4,6u1oRtuoDAlgTOXap7trJlu8Xduj/EsDK+5bbb58zUw=,Va3L257xuywX71/HckPogMlBJcThX6He+akkCaRqr0Y=,0
5,JvtKNwRL9xBP6OXdcgBTgKMhPsBIJdqgFH4lgJ8EgXg=,kJNW3VzhEucFBbFIfHM8eR3osNPKFVTHGNhNmXQpSkE=,1
6,LPGEH3P0EVL44XG9xKv0UNl+f0HIBbEh6F7KbzMUSUI=,eh2PqG0MIhAj1lLRPq3v0oNTp6NStLYKJqmjxV3Pcxw=,0
7,oq4EJseG/aPb4SEYoXGjteoTCpvX1JaA/iDVqpFBL2c=,Zi3DbESRF8fLaxjhpSOFq/E7jfLiUqLV+gwAOvgLYjU=,0
8,IxtJTJmxOAx3G7BbP3Bm8D1jxLnKn1IARY8HHPGh0YI=,E48JY+s/cnVtcxIcPJVnwJgCK6XR9vPW+Ym0OvzhXpM=,0
9,UhF0Bc6aXcXyQ/ZkT6TiLdQOAWUy17pdd93Dwr6gtsw=,ozNT+4kXK4iP++GUsUv23u3vF6WqLQo7I1Iha15y2jw=,1


In [11]:
from sklearn.metrics import roc_auc_score

def roc_metric(pred, target):
    return roc_auc_score(to_np(target.squeeze()), to_np(pred))

learn = collab_learner(dls, y_range=(-0.1, 1.1), metrics=roc_metric)

In [12]:
# learn.fit_one_cycle(12, 1e-1, wd=0.1)

In [13]:
# torch.save(learn.model.state_dict(), '01.pth')
# learn.model.load_state_dict(torch.load('01.pth'))
learn.model.load_state_dict(torch.load('01.pth', map_location=torch.device('cpu')))

/tmp/ipykernel_1333681/1268239533.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  learn.model.load_state_dict(torch.load('01.pth', map_location=torch.device('cpu')))


<All keys matched successfully>

In [14]:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(30756, 50)
  (i_weight): Embedding(359967, 50)
  (u_bias): Embedding(30756, 1)
  (i_bias): Embedding(359967, 1)
)

In [15]:
user_bias_df = pd.DataFrame({
    'msno': learn.dls.classes['msno'],
    'user_bias': to_np(learn.model.u_bias.weight.squeeze()),
})
user_weights_df = pd.DataFrame([
    {
        'msno': msno,
        **{f'user_weight_{i+1}': weight for i, weight in enumerate(weights)}
    }
    for msno, weights in zip(learn.dls.classes['msno'], learn.model.u_weight.weight.tolist())
])
user_embeddings_df = pd.merge(user_bias_df, user_weights_df, on='msno', how='left')

user_embeddings_df

,msno,user_bias,user_weight_1,user_weight_2,user_weight_3,user_weight_4,user_weight_5,user_weight_6,user_weight_7,user_weight_8,...,user_weight_41,user_weight_42,user_weight_43,user_weight_44,user_weight_45,user_weight_46,user_weight_47,user_weight_48,user_weight_49,user_weight_50
0,#na#,-7.006492e-44,7.006492e-44,-7.006492e-44,-7.006492e-44,-7.006492e-44,7.006492e-44,-7.006492e-44,-7.006492e-44,-7.006492e-44,...,-7.006492e-44,7.006492e-44,7.006492e-44,-7.006492e-44,-7.006492e-44,-7.006492e-44,7.006492e-44,7.006492e-44,-7.006492e-44,7.006492e-44
1,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,1.544641e-02,-3.180280e-02,6.770747e-03,3.381065e-02,1.797992e-02,-3.223849e-02,3.264127e-02,1.755419e-02,3.257784e-02,...,3.318285e-02,-3.293496e-03,3.208160e-02,-3.337348e-02,-4.085523e-02,-3.206921e-02,3.342798e-02,2.818430e-02,-3.286837e-02,-3.031654e-02
2,++AH7m/EQ4iKe6wSlfO/xXAJx50p+fCeTyF90GoE9Pg=,3.720637e-02,-4.460219e-02,2.585171e-02,4.374919e-02,2.505669e-02,-4.407157e-02,4.389166e-02,4.288738e-02,4.298264e-02,...,4.406177e-02,-2.838692e-02,4.490560e-02,-4.416827e-02,-4.340399e-02,-4.334666e-02,4.426329e-02,4.322318e-02,-4.215894e-02,-4.381564e-02
3,++e+jsxuQ8UEnmW40od9Rq3rW7+wAum4wooXyZTKJpk=,4.136864e-02,-9.909589e-03,-1.649649e-02,1.038312e-02,-9.236494e-03,-1.017007e-02,1.037820e-02,8.782161e-03,1.027961e-02,...,1.061922e-02,-6.808914e-03,9.964330e-03,-1.041940e-02,2.456308e-04,-1.023243e-02,1.011670e-02,1.043895e-02,-1.023589e-02,-9.596893e-03
4,++xWL5Pbi2CpG4uUugigQahauM0J/sBIRloTNPBybIU=,-1.505532e-02,1.016341e-02,-8.797918e-03,-1.054520e-02,-6.427418e-03,1.059599e-02,-1.043218e-02,-8.649735e-03,-9.509354e-03,...,-1.030097e-02,-6.162331e-03,-1.056312e-02,1.041782e-02,9.660474e-03,9.782305e-03,-1.044357e-02,-9.650602e-03,1.067085e-02,1.128756e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30751,zzTnhToV1XfDIsDJ1YaXM3Q05xV54T8Q0zTJkakXsks=,-3.920763e-02,2.374642e-02,1.390115e-02,-2.376169e-02,6.543166e-03,2.369820e-02,-2.381265e-02,-2.376896e-02,-2.410657e-02,...,-2.390021e-02,1.618624e-02,-2.390879e-02,2.378178e-02,2.618854e-02,2.343843e-02,-2.387727e-02,-2.413770e-02,2.230020e-02,2.218548e-02
30752,zzZBJUYXrb168A4Ff4sA8L2iOH0x4ciKdD6WkV53XaE=,-6.343286e-02,-3.794702e-04,3.986672e-03,-1.570412e-03,3.946521e-03,1.737454e-03,-2.564096e-03,1.357335e-03,-3.280740e-03,...,-2.542384e-03,-6.823685e-03,-7.259926e-03,2.548510e-03,3.721014e-03,3.047608e-03,-3.166670e-03,-1.651816e-03,-2.723103e-03,5.563195e-03
30753,zzompfSaMamqvjyCMYvgUBwYrxh8fHE40z3f73CQoak=,2.919985e-02,1.361843e-02,7.229465e-03,-1.305308e-02,-1.697468e-04,1.291491e-02,-1.299021e-02,-1.621321e-02,-1.698008e-02,...,-1.387063e-02,1.683198e-02,-1.534568e-02,1.223491e-02,-7.598628e-03,1.268087e-02,-1.312982e-02,-1.610591e-02,1.943975e-02,1.306817e-02
30754,zzqc2ja7z10FtSpagYVcAZXg/gPRq7wcDZuNFj+zJSU=,-4.881145e-02,1.524231e-02,1.279851e-02,-1.678162e-02,-5.709238e-04,1.672130e-02,-1.613094e-02,-1.700490e-02,-1.658136e-02,...,-1.698693e-02,1.635220e-02,-1.579126e-02,1.704108e-02,1.395062e-03,1.631640e-02,-1.661794e-02,-1.638646e-02,1.494497e-02,1.884536e-02


In [16]:
song_bias_df = pd.DataFrame({
    'song_id': learn.dls.classes['song_id'],
    'song_bias': to_np(learn.model.i_bias.weight.squeeze()),
})
song_weights_df = pd.DataFrame([
    {
        'song_id': song_id,
        **{f'song_weight_{i+1}': weight for i, weight in enumerate(weights)}
    }
    for song_id, weights in zip(learn.dls.classes['song_id'], learn.model.i_weight.weight.tolist())
])
song_embeddings_df = pd.merge(song_bias_df, song_weights_df, on='song_id', how='left')

song_embeddings_df

,song_id,song_bias,song_weight_1,song_weight_2,song_weight_3,song_weight_4,song_weight_5,song_weight_6,song_weight_7,song_weight_8,...,song_weight_41,song_weight_42,song_weight_43,song_weight_44,song_weight_45,song_weight_46,song_weight_47,song_weight_48,song_weight_49,song_weight_50
0,#na#,-7.006492e-44,-7.006492e-44,7.006492e-44,-7.006492e-44,-7.006492e-44,-7.006492e-44,7.006492e-44,-7.006492e-44,7.006492e-44,...,-7.006492e-44,7.006492e-44,-7.006492e-44,-7.006492e-44,-7.006492e-44,-7.006492e-44,-7.006492e-44,7.006492e-44,7.006492e-44,7.006492e-44
1,+++2AEoM0d8iZTdbnAjUm35bnGKGMXdZJSv4rrWK6JQ=,-5.149811e-03,-1.646146e-03,1.803399e-04,1.609011e-03,1.171259e-03,-1.574734e-03,1.613925e-03,9.709242e-04,1.733579e-03,...,1.595412e-03,-1.912805e-03,1.018357e-03,-1.603969e-03,-4.387705e-04,-9.921337e-04,1.611362e-03,1.448988e-03,-1.388366e-03,-6.787191e-04
2,++/ACCkEN/+VtgrJxEqeRgRmV4y8pcarDJ9T/yRAi1E=,-7.006589e-39,7.006589e-39,-7.006589e-39,7.006589e-39,-7.006589e-39,-7.006589e-39,-7.006589e-39,-7.006589e-39,7.006589e-39,...,-7.006589e-39,7.006589e-39,-7.006589e-39,-7.006589e-39,-7.006589e-39,7.006589e-39,7.006589e-39,-7.006589e-39,-7.006589e-39,-7.006589e-39
3,++/lJNswCU+za2pYB0cWIbGL5UzWIKtfweX20+GImZA=,-7.559547e-05,-1.137183e-05,3.172873e-05,3.941482e-05,4.758242e-05,-3.702887e-05,3.909557e-05,4.932677e-05,3.763581e-05,...,4.834435e-05,-5.548334e-05,4.868666e-05,-4.135095e-05,2.844107e-05,-5.831047e-05,2.279747e-05,5.502540e-05,-6.327069e-05,-2.644087e-05
4,++4/NK5qpbTZWln/6UmykB8cLfRTCCj8E36IKZVzBjM=,-5.605424e-39,-5.605424e-39,-5.605424e-39,-5.605424e-39,-5.605424e-39,5.605424e-39,5.605424e-39,5.605424e-39,-5.605424e-39,...,5.605424e-39,5.605424e-39,-5.605424e-39,-5.605424e-39,5.605424e-39,5.605424e-39,-5.605424e-39,5.605424e-39,5.605424e-39,5.605424e-39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359962,zzvfk6Np17ieMkvG9CQNxdDYQENCVLXuYx5VWrNhCvg=,8.621311e-08,-3.124038e-08,-5.548302e-08,6.160248e-08,5.237249e-09,-6.391538e-08,6.022568e-08,6.279375e-08,6.288931e-08,...,6.917440e-08,-3.609545e-08,-6.721462e-08,-5.549158e-08,4.287873e-08,-7.000056e-08,7.347553e-08,5.892927e-08,-6.720858e-08,-6.123668e-08
359963,zzwCHrZc0TezilVRRTbsiWY6ORNpBq1Zv8GGtHT2xlk=,-4.707767e-27,8.647480e-28,3.825574e-28,1.578456e-27,-1.491436e-27,-1.613684e-27,1.668159e-27,3.684003e-29,3.682709e-28,...,-1.047496e-27,1.220128e-27,6.767657e-28,7.514887e-28,-1.609789e-27,1.124823e-27,3.225607e-28,-4.573997e-28,1.355080e-27,-1.423873e-27
359964,zzwePOl2yF8NLVL7ZfTU2CmtlwnjSruAOpcYiod9iHs=,-9.690684e-04,6.772336e-04,3.665891e-04,-6.142225e-04,5.731036e-04,6.545515e-04,-6.679896e-04,-3.873363e-04,-5.372129e-04,...,-6.060333e-04,-5.640666e-04,-6.558868e-04,6.545478e-04,7.622815e-04,6.675016e-04,-6.392212e-04,-4.958418e-04,2.587390e-04,5.693897e-04
359965,zzxGtSW9L/V3sRNKZbkOgziHtftZ9/oNvETOTY/QHr8=,3.641157e-04,-2.893447e-04,-1.390405e-04,2.891514e-04,8.896524e-05,-2.890686e-04,2.893894e-04,2.826022e-04,2.866190e-04,...,2.893631e-04,-2.396293e-04,2.888475e-04,-2.900666e-04,-2.633114e-04,-2.879529e-04,2.897925e-04,2.891390e-04,-2.872960e-04,-2.877440e-04


In [17]:
song_data_0 = pd.read_csv(path/'songs.csv')
song_data_1 = pd.read_csv(path/'song_extra_info.csv')
members = pd.read_csv(path/'members.csv')

In [18]:
with_song_0 = pd.merge(songs, song_data_0, how='left', on='song_id')
with_song_1 = pd.merge(with_song_0, song_data_1, how='left', on='song_id')
with_members = pd.merge(with_song_1, members, how='left', on='msno')
with_song_embeddings = pd.merge(with_members, song_embeddings_df, how='left', on='song_id')
with_member_embeddings = pd.merge(with_song_embeddings, user_embeddings_df, how='left', on='msno')
all_data = with_member_embeddings

all_data = add_datepart(all_data, 'registration_init_time')
all_data = add_datepart(all_data, 'expiration_date')

all_data.head()

/nix/store/z5kpw9gkd5hza93c8s34fg8hx4hbiy3s-python3-3.11.9-env/lib/python3.11/site-packages/fastai/tabular/core.py:23: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)
/nix/store/z5kpw9gkd5hza93c8s34fg8hx4hbiy3s-python3-3.11.9-env/lib/python3.11/site-packages/fastai/tabular/core.py:23: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)


,msno,song_id,target,song_length,genre_ids,artist_name,composer,lyricist,language,name,...,expiration_Day,expiration_Dayofweek,expiration_Dayofyear,expiration_Is_month_end,expiration_Is_month_start,expiration_Is_quarter_end,expiration_Is_quarter_start,expiration_Is_year_end,expiration_Is_year_start,expiration_Elapsed
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1,206471.0,359,Bastille,Dan Smith| Mark Crew,NaN,52.0,Good Grief,...,1,3,1,False,True,False,True,False,True,0.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,1,284584.0,1259,Various Artists,NaN,NaN,52.0,Lords of Cardboard,...,1,3,1,False,True,False,True,False,True,0.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,1,225396.0,1259,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,NaN,52.0,Hip Hop Is Dead(Album Version (Edited)),...,1,3,1,False,True,False,True,False,True,0.0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,1,255512.0,1019,Soundway,Kwadwo Donkoh,NaN,-1.0,Disco Africa,...,1,3,1,False,True,False,True,False,True,0.0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1,187802.0,1011,Brett Young,Brett Young| Kelly Archer| Justin Ebach,NaN,52.0,Sleep Without You,...,1,3,1,False,True,False,True,False,True,0.0


In [19]:
all_data.to_csv(f'pre_processed.csv', index=False)

In [22]:
list(all_data.columns)

['msno',
 'song_id',
 'target',
 'song_length',
 'genre_ids',
 'artist_name',
 'composer',
 'lyricist',
 'language',
 'name',
 'isrc',
 'city',
 'bd',
 'gender',
 'registered_via',
 'song_bias',
 'song_weight_1',
 'song_weight_2',
 'song_weight_3',
 'song_weight_4',
 'song_weight_5',
 'song_weight_6',
 'song_weight_7',
 'song_weight_8',
 'song_weight_9',
 'song_weight_10',
 'song_weight_11',
 'song_weight_12',
 'song_weight_13',
 'song_weight_14',
 'song_weight_15',
 'song_weight_16',
 'song_weight_17',
 'song_weight_18',
 'song_weight_19',
 'song_weight_20',
 'song_weight_21',
 'song_weight_22',
 'song_weight_23',
 'song_weight_24',
 'song_weight_25',
 'song_weight_26',
 'song_weight_27',
 'song_weight_28',
 'song_weight_29',
 'song_weight_30',
 'song_weight_31',
 'song_weight_32',
 'song_weight_33',
 'song_weight_34',
 'song_weight_35',
 'song_weight_36',
 'song_weight_37',
 'song_weight_38',
 'song_weight_39',
 'song_weight_40',
 'song_weight_41',
 'song_weight_42',
 'song_weight_43

In [23]:
cat_names = ['msno', 'song_id', 'genre_ids', 'artist_name', 'composer', 'lyricist', 'language', 'name', 'isrc', 'city', 'gender', 'registered_via', 'registration_init_timeYear', 'registration_init_timeMonth', 'registration_init_timeWeek', 'registration_init_timeDay', 'registration_init_timeDayofweek', 'registration_init_timeDayofyear', 'registration_init_timeIs_month_end', 'registration_init_timeIs_month_start', 'registration_init_timeIs_quarter_end', 'registration_init_timeIs_quarter_start', 'registration_init_timeIs_year_end', 'registration_init_timeIs_year_start', 'expiration_Year', 'expiration_Month', 'expiration_Week', 'expiration_Day', 'expiration_Dayofweek', 'expiration_Dayofyear', 'expiration_Is_month_end', 'expiration_Is_month_start', 'expiration_Is_quarter_end', 'expiration_Is_quarter_start', 'expiration_Is_year_end', 'expiration_Is_year_start']
cont_names = ['bd', 'song_length', 'registration_init_timeElapsed', 'expiration_Elapsed', 'song_bias', 'song_weight_1', 'song_weight_2', 'song_weight_3', 'song_weight_4', 'song_weight_5', 'song_weight_6', 'song_weight_7', 'song_weight_8', 'song_weight_9', 'song_weight_10', 'song_weight_11', 'song_weight_12', 'song_weight_13', 'song_weight_14', 'song_weight_15', 'song_weight_16', 'song_weight_17', 'song_weight_18', 'song_weight_19', 'song_weight_20', 'song_weight_21', 'song_weight_22', 'song_weight_23', 'song_weight_24', 'song_weight_25', 'song_weight_26', 'song_weight_27', 'song_weight_28', 'song_weight_29', 'song_weight_30', 'song_weight_31', 'song_weight_32', 'song_weight_33', 'song_weight_34', 'song_weight_35', 'song_weight_36', 'song_weight_37', 'song_weight_38', 'song_weight_39', 'song_weight_40', 'song_weight_41', 'song_weight_42', 'song_weight_43', 'song_weight_44', 'song_weight_45', 'song_weight_46', 'song_weight_47', 'song_weight_48', 'song_weight_49', 'song_weight_50', 'user_bias', 'user_weight_1', 'user_weight_2', 'user_weight_3', 'user_weight_4', 'user_weight_5', 'user_weight_6', 'user_weight_7', 'user_weight_8', 'user_weight_9', 'user_weight_10', 'user_weight_11', 'user_weight_12', 'user_weight_13', 'user_weight_14', 'user_weight_15', 'user_weight_16', 'user_weight_17', 'user_weight_18', 'user_weight_19', 'user_weight_20', 'user_weight_21', 'user_weight_22', 'user_weight_23', 'user_weight_24', 'user_weight_25', 'user_weight_26', 'user_weight_27', 'user_weight_28', 'user_weight_29', 'user_weight_30', 'user_weight_31', 'user_weight_32', 'user_weight_33', 'user_weight_34', 'user_weight_35', 'user_weight_36', 'user_weight_37', 'user_weight_38', 'user_weight_39', 'user_weight_40', 'user_weight_41', 'user_weight_42', 'user_weight_43', 'user_weight_44', 'user_weight_45', 'user_weight_46', 'user_weight_47', 'user_weight_48', 'user_weight_49', 'user_weight_50']
dep = 'target'

In [ ]:
to = TabularPandas(all_data, procs=[Categorify, FillMissing,Normalize],
                   cat_names=cat_names,
                   cont_names=cont_names,
                   y_names=dep,
                   splits=RandomSplitter(valid_pct=0.2)(range_of(all_data)))

In [ ]:
test_df = pd.read_csv(path/'test.csv', usecols=('msno', 'song_id', 'id'))

test_df.head()

In [ ]:
test_dl = learn.dls.test_dl(test_df)
output = learn.get_preds(dl=test_dl, with_decoded=True)
output

In [ ]:
probs,_,indexes = output

In [ ]:
sample_submission = pd.read_csv(path/'sample_submission.csv')
sample_submission

In [ ]:
probs = torch.clip(probs, 0, 1)
test_df['target'] = probs

sub_df = test_df[['id','target']]
sub_df.to_csv(f'submission.csv', index=False)

In [ ]:
!head submission.csv

In [ ]:
if not iskaggle:
    from kaggle import api
    api.competition_submit_cli('submission.csv', 'kkbox-recommendation-3', comp)

In [5]:
if not iskaggle:
    push_notebook('alexchalk', 'poisonous-mushrooms-submission-05',
                  title='Poisonous Mushrooms Submission 05',
                  file='02.ipynb',
                  competition=comp, private=True, gpu=False)

Kernel version 1 successfully pushed.  Please check progress at https://www.kaggle.com/code/alexchalk/poisonous-mushrooms-submission-05
